In [1]:
import os.path as osp

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid, MNISTSuperpixels
from torch_geometric.data import DataLoader


import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, ChebConv


In [2]:
dataset = 'Mnist'
path = '../data/geometric/MNIST'
dataset = MNISTSuperpixels(path, dataset, T.Distance())
loader = DataLoader(dataset, batch_size=32, shuffle=True)


# dataset = Planetoid(path, dataset, T.NormalizeFeatures())
#data = dataset[0]

In [3]:
print(dataset.num_features)

1


In [28]:
# class Net(torch.nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = GCNConv(dataset.num_features, 16, improved=False)
#         self.conv2 = GCNConv(16, dataset.num_classes, improved=False)
#         # self.conv1 = ChebConv(data.num_features, 16, K=2)
#         # self.conv2 = ChebConv(16, data.num_features, K=2)
        
#     def forward(self):
#         x, edge_index = data.x, data.edge_index
#         x = F.relu(self.conv1(x, edge_index))
#         x = F.dropout(x, training=self.training)
#         x = self.conv2(x, edge_index)
#         return F.log_softmax(x, dim=1)


class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, 16, improved=False)
        self.conv2 = GCNConv(16, dataset.num_classes, improved=False)
        # self.conv1 = ChebConv(data.num_features, 16, K=2)
        # self.conv2 = ChebConv(16, data.num_features, K=2)
        
        self.deconv1 = GCNConv(dataset.num_classes, 16, improved=False)
        self.deconv2 = GCNConv(16, dataset.num_features, improved=False)
        
    def encode(self, x, edge_index):
        
        x = F.relu(self.conv1(x, edge_index))
        #x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        
        return x, edge_index
    
    def decode(self, x, edge_index):
        x = F.relu(self.deconv1(x, edge_index))
        #x = F.dropout(x, training=self.training)
        x = self.deconv2(x, edge_index)
        
        return x
        
        
    def forward(self):
        
        x, edge_index = data.x, data.edge_index
        
        encoded, idx = self.encode(x, edge_index)
        decoded = self.decode(encoded, idx)
        
        return decoded

In [29]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

numEpochs = 2

In [30]:
for epoch in range(1, numEpochs+1):
    for step, batch in enumerate(loader):
        if step < 20:
            
            data = batch.to(device)
            model.train()
            optimizer.zero_grad()

            loss = nn.MSELoss()
            target = data
            
#             print(target['x'])
#             print(model())
            
            output = loss(model(), target['x'])
            output.backward()


#             F.binary_cross_entropy(model(), target['x']).backward()
            optimizer.step()
        

            print('Epoch: ', epoch, 'train loss: ', output.item())
        else: 
            break
    

#             # plotting decoded image (second row)
#             _, decoded_data = autoencoder(view_data)
#             for i in range(N_TEST_IMG):
#                 a[1][i].clear()
#                 a[1][i].imshow(np.reshape(decoded_data.data.numpy()[i], (28, 28)), cmap='gray')
#                 a[1][i].set_xticks(()); a[1][i].set_yticks(())
#             plt.draw(); plt.pause(0.05)

Epoch:  1 train loss:  0.19061681628227234
Epoch:  1 train loss:  0.15084707736968994
Epoch:  1 train loss:  0.13869276642799377
Epoch:  1 train loss:  0.12381277233362198
Epoch:  1 train loss:  0.12354801595211029
Epoch:  1 train loss:  0.11772618442773819
Epoch:  1 train loss:  0.10465120524168015
Epoch:  1 train loss:  0.09563049674034119
Epoch:  1 train loss:  0.09878691285848618
Epoch:  1 train loss:  0.10717673599720001
Epoch:  1 train loss:  0.11034782975912094
Epoch:  1 train loss:  0.11038356274366379
Epoch:  1 train loss:  0.11148583889007568
Epoch:  1 train loss:  0.10845232754945755
Epoch:  1 train loss:  0.10239468514919281
Epoch:  1 train loss:  0.09956809133291245
Epoch:  1 train loss:  0.10118277370929718
Epoch:  1 train loss:  0.09562849253416061
Epoch:  1 train loss:  0.10060091316699982
Epoch:  1 train loss:  0.10658057779073715
Epoch:  2 train loss:  0.09860781580209732
Epoch:  2 train loss:  0.10430910438299179
Epoch:  2 train loss:  0.10513541102409363
Epoch:  2 t

In [7]:
# def train():
#     model.train()
#     optimizer.zero_grad()
    
#     F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    
#     loss = nn.BCELoss()
#     target = data.x[data.train_mask]
#     print(target)
#     print(model()[data.train_mask])
    
#     output = loss(model()[data.train_mask], target)
#     output.backward()
    
    
#     torch.nn.BCELoss(model()[data.train_mask], data.x[data.train_mask]).backward()
    
#     optimizer.step()

# def test():
#     model.eval()
#     logits, accs = model(), []
#     for _, mask in data('train_mask', 'val_mask', 'test_mask'):
#         pred = logits[mask].max(1)[1]
#         acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
#         accs.append(acc)
#     return accs

# best_val_acc = test_acc = 0
# for epoch in range(1, 50):
#     train()
#     train_acc, val_acc, tmp_test_acc = test()
#     if val_acc > best_val_acc:
#         best_val_acc = val_acc
#         test_acc = tmp_test_acc
#     log = 'Epoch: {:03d}, Train: {:.4f}, Val: {:.4f}, Test: {:.4f}'
#     print(log.format(epoch, train_acc, best_val_acc, test_acc))


In [32]:
print(len(target))

6
